In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from torch import load, Tensor, save
import random
import pickle

In [2]:
relations_file_path = '../data/br/firefox/firefox_pairs.csv'

relations_openoffice = pd.read_csv(relations_file_path, index_col='issue_id')


reports_val_file_path = '../data/splits/firefox/firefox_val.csv'
reports_test_file_path = '../data/splits/firefox/firefox_test.csv'

reports_openoffice_val = pd.read_csv(reports_val_file_path, index_col='bug_id')
reports_openoffice_test = pd.read_csv(reports_test_file_path, index_col='bug_id')

reports_openoffice_appended = pd.concat([reports_openoffice_val, reports_openoffice_test])

In [3]:
print(len(reports_openoffice_val))
print(len(reports_openoffice_test))
print(len(reports_openoffice_appended))

24321
34745
59066


In [4]:
filtered_relations = relations_openoffice.loc[relations_openoffice.index.intersection(reports_openoffice_appended.index)]

In [5]:
len(relations_openoffice)

92651

In [6]:
len(filtered_relations)

47167

In [8]:
filtered_relations.head(20)

,duplicate
10954,NaN
14871,243500;410103;505684;515027;528678
54746,NaN
70171,NaN
75119,NaN
83003,NaN
84106,NaN
91774,NaN
92073,NaN
93789,354686;394099


In [9]:
relations = {}

for key in filtered_relations.index:
    
    if key in reports_openoffice_val.index:
        index = key
        try:
            duplicates_id_whole_dataset = [int(id) for id in filtered_relations.loc[index].values[0].split(';')]
            duplicates_id = [x for x in duplicates_id_whole_dataset if x in reports_openoffice_val.index]
        except:
            duplicates_id = []

        if len(duplicates_id) > 0 and relations.get(index) is None:
            relations[index] = duplicates_id


In [11]:
len(relations) / 100

36.61

In [12]:
samples_reports = {}
samples_relations = []

num_of_samples = int(len(relations) / 100)

for i in range(0, num_of_samples):
    random_realations = random.sample(relations.items(), 100)

    random_realations = dict(random_realations)
    samples_relations.append(random_realations)

    for key in random_realations.keys():
        relations.pop(key)


counter = 1

for sample_relations in samples_relations:
    target_ids = set([x for x in sample_relations.keys()])

    for ids_list in sample_relations.values():
        for value in ids_list:
            target_ids.add(value)

    filtered_df = reports_openoffice_appended.loc[reports_openoffice_appended.index.intersection(target_ids)][['description', 'short_desc']]
    selected_relations_series = pd.Series(sample_relations)
    selected_relations_df = pd.DataFrame({'duplicates': selected_relations_series})
    selected_relations_df.index.name = "issue_id"

    samples_reports[counter] = {
        "relations": selected_relations_df,
        "reports": filtered_df
    }
    
    counter += 1



C:\Users\lpros\AppData\Local\Temp\ipykernel_14132\519863780.py:7: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_realations = random.sample(relations.items(), 100)


In [13]:
print(len(samples_relations[0].keys()))
print(len(samples_relations[1].keys()))
print(len(samples_relations[2].keys()))

100
100
100


In [14]:
print(len(samples_reports[1]["reports"]))
print(len(samples_reports[2]["reports"]))
print(len(samples_reports[3]["reports"]))
print(len(samples_reports[4]["reports"]))
print(len(samples_reports[5]["reports"]))
        

436
418
425
428
407


In [15]:
samples_reports[1]["relations"]

,duplicates
issue_id,
281384,[345882]
638737,[505117]
361551,"[356172, 358004, 358457, 359180, 360337, 36051..."
375341,[369882]
376250,"[466454, 482840, 500356]"
...,...
370787,[410325]
426375,[424200]
402465,[306965]


In [17]:
file_path = "data/firefox_samples.pkl"
with open(file_path, 'wb') as f:
    pickle.dump(samples_reports, f)